In [1]:
import os
import sys

import pandas as pd

In [2]:
source_df = pd.read_csv("question_c_output/output.csv")
source_df.head()

,domain name,status,remote_ip,remote_port,time_appconnect,time_connect,time_namelookup,time_pretransfer,time_redirect,time_starttransfer,time_total
0,google.com,ok,216.58.212.132,443.0,0.0s,0.039137s,0.017335s,0.039238s,0.0s,0.083712s,0.083847s
1,facebook.com,ok,185.60.216.35,443.0,0.0s,0.015666s,0.011418000000000001s,0.015778999999999998s,0.0s,0.12213099999999999s,0.15120999999999998s
2,youtube.com,ok,216.58.207.46,443.0,0.0s,0.034769s,0.011472s,0.034894s,0.0s,0.068s,0.673338s
3,microsoft.com,timeout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,twitter.com,ok,104.244.42.129,443.0,0.0s,0.0s,0.016481s,0.0s,0.0s,0.0s,0.086354s


In [3]:
regex_df = pd.read_csv("reference/csv/regexes.csv", header=None, names=['provider', 'reg_match','c'])
regex_df.drop("c", axis=1,inplace=True)
regex_df['provider'] = regex_df['provider'].str.extract('([a-zA-Z]+)', expand=False)
regex_df['reg_match'] = regex_df['reg_match'].str[:-1]
regex_df.head()

,provider,reg_match
0,Cedexis,re.compile(r'\.cedexis\.net')
1,Akamai,re.compile(r'\.akamai\.net|\.edgesuite\.net|\...
2,Facebook,re.compile(r'\.facebook\.com|\.facebook\.net|...
3,Microsoft,re.compile(r'\.azureedge\.net|\.azure\.micros...
4,Google,re.compile(r'googlehosted\.com|\.google\.|goo...


In [4]:
asns_df = pd.read_csv("reference/csv/asns.csv",names=["col1", "col2", "col3"], skiprows=1)
asns_df.head()

,col1,col2,col3
0,Cedexis;RIPE;CEDEXIS - Cedexis Inc,US;53847,NaN
1,Akamai;PeeringDB;Akamai Technologies;20940,NaN,NaN
2,Akamai;PeeringDB;Akamai Prolexic DDoS Mitigati...,NaN,NaN
3,Akamai;RIPE;AKAMAI - Akamai Technologies,Inc.,US;12222
4,Akamai;RIPE;AKAMAI-AS - Akamai Technologies,Inc.,US;16625


In [5]:
asns_df['provider'], asns_df['lookup_type'], asns_df['asn'] = asns_df['col1'].str.split(';', 2).str
asns_df.drop("col1", axis=1,inplace=True)
asns_df.head()

,col2,col3,provider,lookup_type,asn
0,US;53847,NaN,Cedexis,RIPE,CEDEXIS - Cedexis Inc
1,NaN,NaN,Akamai,PeeringDB,Akamai Technologies;20940
2,NaN,NaN,Akamai,PeeringDB,Akamai Prolexic DDoS Mitigation;32787
3,Inc.,US;12222,Akamai,RIPE,AKAMAI - Akamai Technologies
4,Inc.,US;16625,Akamai,RIPE,AKAMAI-AS - Akamai Technologies


In [6]:
as_all_df = pd.merge(asns_df, regex_df, on='provider')
as_all_df

,col2,col3,provider,lookup_type,asn,reg_match
0,US;53847,NaN,Cedexis,RIPE,CEDEXIS - Cedexis Inc,re.compile(r'\.cedexis\.net')
1,NaN,NaN,Akamai,PeeringDB,Akamai Technologies;20940,re.compile(r'\.akamai\.net|\.edgesuite\.net|\...
2,NaN,NaN,Akamai,PeeringDB,Akamai Prolexic DDoS Mitigation;32787,re.compile(r'\.akamai\.net|\.edgesuite\.net|\...
3,Inc.,US;12222,Akamai,RIPE,AKAMAI - Akamai Technologies,re.compile(r'\.akamai\.net|\.edgesuite\.net|\...
4,Inc.,US;16625,Akamai,RIPE,AKAMAI-AS - Akamai Technologies,re.compile(r'\.akamai\.net|\.edgesuite\.net|\...
...,...,...,...,...,...,...
308,NO;39197,NaN,Telenor,RIPE,TELENOR-NORGE NORWAY,re.compile(r'\.cdntel\.net')
309,MM;133385,NaN,Telenor,RIPE,TELENORMYANMAR-AS Telenor Myanmar,re.compile(r'\.cdntel\.net')
310,ES;204785,NaN,Telenor,RIPE,TELENORTSOLUCION,re.compile(r'\.cdntel\.net')
311,NaN,NaN,CDNvideo,PeeringDB,CDNvideo;57363,re.compile(r'\.cdnvideo.ru|\.cdnvideo.net')


In [7]:
#### change here
to_repl = as_all_df.reg_match.values.tolist()
vals = as_all_df.asn.values.tolist()
source_df['asn'] = source_df['domain name'].replace(to_repl, vals, regex=True)

In [8]:
source_df

,domain name,status,remote_ip,remote_port,time_appconnect,time_connect,time_namelookup,time_pretransfer,time_redirect,time_starttransfer,time_total,asn
0,google.com,ok,216.58.212.132,443.0,0.0s,0.039137s,0.017335s,0.039238s,0.0s,0.083712s,0.083847s,google.com
1,facebook.com,ok,185.60.216.35,443.0,0.0s,0.015666s,0.011418000000000001s,0.015778999999999998s,0.0s,0.12213099999999999s,0.15120999999999998s,facebook.com
2,youtube.com,ok,216.58.207.46,443.0,0.0s,0.034769s,0.011472s,0.034894s,0.0s,0.068s,0.673338s,youtube.com
3,microsoft.com,timeout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,microsoft.com
4,twitter.com,ok,104.244.42.129,443.0,0.0s,0.0s,0.016481s,0.0s,0.0s,0.0s,0.086354s,twitter.com
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,shareholder.com,resolve host error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,shareholder.com
9996,noxinfluencer.com,ok,104.18.26.59,443.0,0.0s,0.0s,0.016062s,0.0s,0.0s,0.0s,7.502421000000001s,noxinfluencer.com
9997,jarir.com,ok,104.16.41.5,443.0,0.0s,0.0s,0.232406s,0.0s,0.0s,0.0s,7.732135s,jarir.com
9998,investorplace.com,ok,104.18.230.99,443.0,0.0s,0.0s,0.23681300000000002s,0.0s,0.0s,0.0s,7.736503999999999s,investorplace.com


In [10]:
###sCount number of websites for each asn
source_df = source_df.groupby('asn')['status'].apply(lambda x: (x=='ok').sum()).reset_index(name='num_http3_websites').sort_values('num_http3_websites', ascending=0)
source_df

,asn,num_http3_websites
0,000webhost.com,1
7593,sheypoor.com,1
3341,game8.jp,1
7589,shein.com,1
7588,sheffield.ac.uk,1
...,...,...
3964,hindustantimes.com,0
3963,hindawi.com,0
3962,hinative.com,0
3961,himado.in,0


In [11]:
source_df.to_csv("num_http3_websites.csv")

#### e) Order the collected timing metrics by the sequential order of their corresponding events (i.e., DNS lookup happens before y, etc.). Calculate the starting/end times and duration of the events; note that some metrics include other metrics, so you will need to calculate deltas in some cases. Draw boxplots over all measurements for each of the events/metrics (cf. Assignment 02); you can use the description of the events/metrics as labels for your x-axis (e.g., DNS lookup, ...). Submit your answers/descriptions, analysis and plotting scripts, and plot.